In [1]:
# For conveniance: 
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path
from dotenv import load_dotenv

working_dir = Path.cwd()
while working_dir.name != 'CausalMTR-BC':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'CausalMTR-BC' not found")
os.chdir(working_dir)

# Load env variables from config/.env 
dotenv_path = Path("configs/.env")
load_dotenv(dotenv_path=dotenv_path)

from gpudrive.env.config import EnvConfig, SceneConfig
from gpudrive.env.env_torch import GPUDriveTorchEnv
from gpudrive.visualize.utils import img_from_fig
from gpudrive.env.dataset import SceneDataLoader

In [2]:
DYNAMICS_MODEL = "delta_local" # "delta_local" / "state" / "classic"
DATA_PATH = "data/processed/examples" # Your data path
MAX_NUM_OBJECTS = 64
NUM_ENVS = 4
DEVICE = "cuda:1"

# Configs
env_config = EnvConfig(dynamics_model=DYNAMICS_MODEL)

In [3]:
# Make dataloader
data_loader = SceneDataLoader(
    root="gpudrive/data/processed/examples", # Path to the dataset
    batch_size=NUM_ENVS, # Batch size, you want this to be equal to the number of worlds (envs) so that every world receives a different scene
    dataset_size=NUM_ENVS, # Total number of different scenes we want to use
    sample_with_replacement=False, 
    seed=42, 
    shuffle=True,   
)

print("Making the environment")

# Make environment
env = GPUDriveTorchEnv(
    config=env_config,
    data_loader=data_loader,
    max_cont_agents=MAX_NUM_OBJECTS, # Maximum number of agents to control per scenario
    device="cuda", 
    action_type="continuous" # "continuous" or "discrete"
)

Making the environment
Compiling GPU engine code:
/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/memory.cpp
/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/state.cpp
/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/crash.cpp
/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/consts.cpp
/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/taskgraph.cpp
/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/taskgraph_utils.cpp
/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/sort_archetype.cpp
/CausalMTR-BC/gpudrive/external/madrona/src/mw/device/host_print.cpp
/CausalMTR-BC/gpudrive/external/madrona/src/mw/../common/hashmap.cpp
/CausalMTR-BC/gpudrive/external/madrona/src/mw/../common/navmesh.cpp
/CausalMTR-BC/gpudrive/external/madrona/src/mw/../core/base.cpp
/CausalMTR-BC/gpudrive/external/madrona/src/mw/../physics/physics.cpp
/CausalMTR-BC/gpudrive/external/madrona/src/mw/../physics/geo.cpp
/CausalMTR-BC/gpudrive/external/madrona/src/mw/../physics/xpbd.cpp
/Ca

/CausalMTR-BC/gpudrive/src/level_gen.cpp(283): warning #177-D: function "madrona_gpudrive::createFloorPlane" was declared but never referenced
  static void createFloorPlane(Engine &ctx)
              ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"





In [4]:
from cmtr_bc.waymo_dataset import WaymoDataset
import yaml 
from easydict import EasyDict
import logging
import torch
import numpy as np

In [5]:
def getconfig(cfg_file):
    with open(cfg_file, 'r') as f:
        try:
            config = yaml.load(f, Loader=yaml.FullLoader)
        except:
            config = yaml.load(f)

    return EasyDict(config)

logger = logging.getLogger(__name__)

config = getconfig("configs/config.yaml")
waymo_dataset = WaymoDataset(config.DATA_CONFIG, test_mode=1, logger=logger)

In [6]:
from mtr.models import model as model_utils
from cmtr_bc.waymo_iterator import TrajectoryIterator
from torch.utils.data import DataLoader

/opt/miniforge3/envs/gpudrive/lib/python3.11/site-packages/colt5_attention/triton_coor_descent.py:314: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/opt/miniforge3/envs/gpudrive/lib/python3.11/site-packages/colt5_attention/triton_coor_descent.py:410: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


In [7]:
traj_data_loader = TrajectoryIterator(env, data_loader, cmtr=True, waymo_dataset=waymo_dataset, prior_frame=20, future_frame=40, simple=False)
train_iterator = iter(DataLoader(traj_data_loader, batch_size=2, collate_fn=traj_data_loader.collate_batch))

In [8]:
MODEL_DEVICE = "cuda:1"
model_cfg = config.MODEL
model_cfg.CONTEXT_ENCODER.NUM_INPUT_ATTR_AGENT = 38
model_cfg.CONTEXT_ENCODER.NUM_ATTN_LAYERS = 1
model_cfg.CONTEXT_ENCODER.NUM_CHANNEL_IN_MLP_AGENT = 64
model_cfg.MOTION_DECODER.NUM_DECODER_LAYERS = 1
model_cfg.MOTION_DECODER.NUM_FUTURE_FRAMES = 40
torch.cuda.set_device(1)
model = model_utils.MotionTransformer(config=model_cfg).to(MODEL_DEVICE)

In [16]:
model.set_epoch(20)

In [17]:
batch_dict, actions, _, _ = next(train_iterator)
loss, tb_dict, disp_dict, batch_dict = model(batch_dict)

In [18]:
tb_dict

{'loss_layer0': 4622.67578125,
 'loss_layer0_reg_gmm': 4168.06298828125,
 'loss_layer0_reg_vel': 92.6409912109375,
 'loss_layer0_cls': 4.208978176116943,
 'ade_TYPE_VEHICLE_layer_0': 2.320590058962504,
 'ade_TYPE_PEDESTRIAN_layer_0': -0.0,
 'ade_TYPE_CYCLIST_layer_0': -0.0,
 'loss_dense_prediction': 3894.41748046875,
 'loss': 8517.11328125,
 'mask_predictor_loss': 0.019500000402331352}

In [ ]:
'''
Because of annealment, mask_predictor loss is dependent on epoch - check mask_predictor_default.yaml for 
annealment config
'''

In [ ]:
'''
Because of annealment, mask_predictor loss is dependent on epoch - check mask_predictor_default.yaml for 
annealment config
'''